In [43]:
import pandas as pd
import re
import json
import pandas as pd
import requests
from tqdm import tqdm
import numpy as np

saramin_df = pd.read_csv('saramin_df.csv', encoding='utf-8')
saramin_df.head()

,회사명,직무,링크,공고내용,회사주소
0,(주)이음인터렉티브,데이터 사이언티스트,https://www.saramin.co.kr/zf_user/jobs/relay/v...,2023 신규채용\n# 데이터사이언티스트\n# 데이터엔지니어 \n㈜이음인터렉티브는 ...,"(06621) 서울 서초구 강남대로 369, 12층\n신분당선 강남역에서 200m 이내"
1,데이콘(주),데이터 사이언티스트,https://www.saramin.co.kr/zf_user/jobs/relay/v...,데이터 사이언티스트 인턴 채용(채용 시 조기 마감)\n\n\n\n\n모집부문 및 상...,"(07237) 서울 영등포구 은행로 3 익스콘벤처타워, 901호(여의도동)\n서울 ..."
2,(주)위키드스톰,데이터 사이언티스트,https://www.saramin.co.kr/zf_user/jobs/relay/v...,데이터 사이언티스트 경력직 채용\n\n\n\n\n모집부문 및 상세내용\n데이터 사이...,(07788) 서울 강서구 마곡중앙로 161-8 두산더랜드파크 8층 819호(마곡동...
3,(주)카비,데이터 사이언티스트,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"◆ CARVI News\n\n[ZDNet Korea] 카비-교통장애인협회, AI로 ...",(153-788) 서울 금천구 가산동 벽산디지털밸리5차 301호
4,(주)핀커스코리아,데이터 사이언티스트,https://www.saramin.co.kr/zf_user/jobs/relay/v...,[기업소개] \n- 이커머스에 특화된 AI/빅데이터 플랫폼 기업\n- 차별화 된 비...,(135-080) 서울 강남구 역삼동 736-42 예동빌딩 8층\n서울 2호선 역삼...


In [44]:
road_df = []

for i in range(len(saramin_df)):
    road_ = saramin_df['회사주소'][i].split(sep='\n')[0]
    pattern = r'\([^)]*\)'      # 지번의 경우 앞에 우편번호 들어가면 좌표로 변환이 안되어서 괄호 내에 내용 삭제하였습니다.
    road = re.sub(pattern=pattern, repl='', string=road_)
    road_df.append(road)
    
saramin_df['road_address'] = road_df

In [45]:
# 브이월드 openapi를 활용하여 주소를 좌표로 변환하였습니다.

index = np.arange(5)

url = 'http://api.vworld.kr/req/address?'
params = 'service=address&request=getcoord&version=2.0&crs=epsg:4326&refine=true&simple=false&format=json&type='
address = '&address='
keys = '&key='
primary_key = '359D1A26-3D4E-308E-8AFF-9EE5706D3133'


def request_geo_road(road):
    if road=='None':
        json_data == 'NOT_FOUND'
    else:
        page = requests.get(url+params+'ROAD'+address+road+keys+primary_key)
        json_data = page.json()
    return json_data

def request_geo_parcel(road):
    if road=='None':
        json_data == 'NOT_FOUND'
    else:
        page = requests.get(url+params+'PARCEL'+address+road+keys+primary_key)
        json_data = page.json()
    return json_data

def extraction_geo_road(road_data):
    geocode = pd.DataFrame(columns = ['회사명','road_address', 'x', 'y'])
    none = None
    ct = 1
    print(ct)
    ct += 1
    for idx, road in tqdm(zip(road_data.index ,road_data['road_address'])):
        name = str(road_data['회사명'][idx])
        if len(str(road)) <= 5:
            geocode = geocode.append(
                    pd.DataFrame({'회사명':name,
                    'road_address':road,
                    'x':none,
                    'y':none},
                    index=[idx]))
            continue

        json_data = request_geo_road(road)

        if json_data['response']['status'] == 'NOT_FOUND' or json_data['response']['status'] == 'ERROR':
            geocode = geocode.append(
                    pd.DataFrame({'회사명':name,
                    'road_address':road,
                    'x':none,
                    'y':none},
                    index=[idx]))
            continue

        x = json_data['response']['result']['point']['x']
        y = json_data['response']['result']['point']['y']

        geocode = geocode.append(
            pd.DataFrame({'회사명':name,
                    'road_address':road,
                    'x':float(x),
                    'y':float(y)},
                    index=[idx]))
    return geocode

def extraction_geo_parcel(parcel_data):
    geocode = pd.DataFrame(columns = ['회사명','road_address', 'x', 'y'])
    none = None
    ct = 1
    print(ct)
    ct += 1
    for idx, road in tqdm(zip(parcel_data.index ,parcel_data['road_address'])):
        name = str(parcel_data['회사명'][idx])
        if len(str(road)) <= 5:
            geocode = geocode.append(
                    pd.DataFrame({'회사명':name,
                    'road_address':road,
                    'x':none,
                    'y':none},
                    index=[idx]))
            continue

        json_data = request_geo_parcel(road)

        if json_data['response']['status'] == 'NOT_FOUND' or json_data['response']['status'] == 'ERROR':
            geocode = geocode.append(
                    pd.DataFrame({'회사명':name,
                    'road_address':road,
                    'x':none,
                    'y':none},
                    index=[idx]))
            continue

        x = json_data['response']['result']['point']['x']
        y = json_data['response']['result']['point']['y']

        geocode = geocode.append(
            pd.DataFrame({'회사명':name,
                    'road_address':road,
                    'x':float(x),
                    'y':float(y)},
                    index=[idx]))
    return geocode



In [48]:
road_data = pd.DataFrame(road_df, columns = ['road_address'])
road_data['회사명'] = saramin_df['회사명']

coordinate = extraction_geo_road(road_data)

1


0it [00:00, ?it/s]C:\Users\Yun\AppData\Local\Temp\ipykernel_38084\534466685.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  geocode = geocode.append(
1it [00:00,  9.79it/s]C:\Users\Yun\AppData\Local\Temp\ipykernel_38084\534466685.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  geocode = geocode.append(
C:\Users\Yun\AppData\Local\Temp\ipykernel_38084\534466685.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  geocode = geocode.append(
3it [00:00, 10.71it/s]C:\Users\Yun\AppData\Local\Temp\ipykernel_38084\534466685.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  geocode = geocode.append(
C:\Users\Yun\AppData\Local\Temp\ip

In [49]:
parcel_data = pd.DataFrame(road_df, columns = ['road_address'])
parcel_data['회사명'] = saramin_df['회사명']

coordinates = extraction_geo_parcel(parcel_data)

1


0it [00:00, ?it/s]C:\Users\Yun\AppData\Local\Temp\ipykernel_38084\534466685.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  geocode = geocode.append(
C:\Users\Yun\AppData\Local\Temp\ipykernel_38084\534466685.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  geocode = geocode.append(
2it [00:00, 14.18it/s]C:\Users\Yun\AppData\Local\Temp\ipykernel_38084\534466685.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  geocode = geocode.append(
C:\Users\Yun\AppData\Local\Temp\ipykernel_38084\534466685.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  geocode = geocode.append(
4it [00:00, 13.39it/s]C:\Users\Yun\AppData\Local\Temp\ip

In [50]:
drop_na_coordinate = coordinate.dropna(axis=0)
drop_na_coordinates = coordinates.dropna(axis=0)

coordinate_data = pd.concat([drop_na_coordinates, drop_na_coordinate])
coordinate_data.reset_index(drop=False, inplace=False)
coordinate_data.to_csv("coordinate.csv", index = False)